<a href="https://colab.research.google.com/github/Xcalibur-hub/machine-learning/blob/main/Plant_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.applications import VGG16, VGG19, InceptionV3, ResNet50

# Mount Google Drive
drive.mount('/content/drive')

def count_images(directory):
    return sum(len(files) for _, _, files in os.walk(directory))

def prepare_data(train_dir, test_dir, target_size=(224, 224), batch_size=32):
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=target_size, batch_size=batch_size)
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=target_size, shuffle=True, batch_size=batch_size)

    return train_generator, test_generator

def build_model(base_model, num_classes):
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=predictions)

def train_model(model, train_generator, validation_generator, epochs=5):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator,
                        callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)])
    return history

# Paths
train_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/Plant Leafs/train"
test_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/Plant Leafs/val"

# Image counts
num_classes = len(glob.glob(train_dir + "/*"))
train_samples = count_images(train_dir)
test_samples = count_images(test_dir)

print(f"{num_classes} Classes")
print(f"{train_samples} Train images")
print(f"{test_samples} Test images")

# Prepare data
train_generator, test_generator = prepare_data(train_dir, test_dir)

# Build and train models
base_models = [VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
               VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
               InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
               ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))]

for base_model in base_models:
    model = build_model(base_model, num_classes)
    history = train_model(model, train_generator, test_generator)

# Save model example
model.save('/content/drive/MyDrive/Colab Notebooks/Model/my_model.h5')

# Load and predict example
def prepare_img(img_path):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

img_url = '/path/to/image.jpg'
model = load_model('/content/drive/MyDrive/Colab Notebooks/Model/my_model.h5')
result = model.predict(prepare_img(img_url))
predicted_class = np.argmax(result)
print(f"Predicted class: {predicted_class}")
